In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
! mkdir -p data
! wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet -O data/fhv_tripdata_2021-01.parquet
! wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet -O data/fhv_tripdata_2021-02.parquet

--2022-05-22 22:32:35--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.105.36
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.105.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘data/fhv_tripdata_2021-01.parquet’

data/fhv_tripdata_2 100%[===================>]  11.33M  23.9MB/s    in 0.5s    

2022-05-22 22:32:35 (23.9 MB/s) - ‘data/fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]

--2022-05-22 22:32:36--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.105.36
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.105.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘data/fhv_tripdata_2021-02.pa

In [4]:
df = pd.read_parquet("data/fhv_tripdata_2021-01.parquet")

print("Q1: January total records: ", len(df))

df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
print("Q2: average duration in January: ", round(df.duration.mean(), 2))

Q1: January total records:  1154112
Q2: average duration in January:  19.17


In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
        df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].fillna(-1)
    df[categorical] = df[categorical].astype(str)
    
    return df

df_train = read_dataframe('./data/fhv_tripdata_2021-01.parquet')
print("train after filtering: ", len(df_train))
df_val = read_dataframe('./data/fhv_tripdata_2021-02.parquet')

train after filtering:  1109826


In [6]:
total_filtered = len(df_train)
pu_location_missing = len(df_train[df_train['PUlocationID'] == "-1.0"])
print("Q3: fraction of PU location missing: ", pu_location_missing/total_filtered)


Q3: fraction of PU location missing:  0.8352732770722617


In [7]:
categorical = ['PUlocationID', 'DOlocationID']
dv = DictVectorizer()
train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
print("Q4: dimension of the matrix: ", X_train.shape[1])


Q4: dimension of the matrix:  525


In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [10]:
y_train_pred = lr.predict(X_train)
print("Q5: RMSE on training: ", mean_squared_error(y_train, y_train_pred, squared=False))

y_val_pred = lr.predict(X_val)
print("Q6: RMSE on validation: ", mean_squared_error(y_val, y_val_pred, squared=False))

Q5: RMSE on training:  10.528519107213159
Q6: RMSE on validation:  11.01428321291995
